# Modelos Redes Neurais
## Cabeçalho
Criado por: Danilo Steckelberg <br>
Criado para: Trabalho de conclusão de curso IBM Deep Learning (https://www.coursera.org/learn/deep-learning-reinforcement-learning) <br>
Criado em: 2023-01-19 <br>
Direitos dos dados: Evi Brasil <br>

## Carregar dados
Origem dos dados: Rotina em R [File](file://C:\\Users\\danil\\Documents\\Danilo\\Evi%20Data%20Pipeline\\evi_dados\\tcc_mba_dsa\\papers\\OUT_comparacao_modelos.R)

Tratamento dos dados realizado nas rotinas em R

In [55]:
import pandas as pd
import numpy as np
dados_entrada = pd.read_csv('.\\data\\nn_input.csv')

In [2]:
import tensorflow as tf
tf.random.set_seed(42)

In [3]:
num_layers = 3
min_nodes_per_layer = 64
max_nodes_per_layer = 256
node_step_size = 64

In [4]:
node_options = list(range(
    min_nodes_per_layer,
    max_nodes_per_layer + 1,
    node_step_size
))

In [6]:
import itertools

two_layer_possibilities = [node_options, node_options]

list(itertools.product(*two_layer_possibilities))

[(64, 64),
 (64, 128),
 (64, 192),
 (64, 256),
 (128, 64),
 (128, 128),
 (128, 192),
 (128, 256),
 (192, 64),
 (192, 128),
 (192, 192),
 (192, 256),
 (256, 64),
 (256, 128),
 (256, 192),
 (256, 256)]

In [9]:
layer_possibilities = [node_options] * num_layers
layer_node_permutations = list(itertools.product(*layer_possibilities))

In [8]:
for permutation in layer_node_permutations[:2]:
    for nodes_at_layer in permutation:
        print(nodes_at_layer)
    print()

64
64
64

64
64
128



In [10]:
models = []

for permutation in layer_node_permutations:
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = (12,)))
    model_name = ''

    for nodes_at_layers in permutation:
        model.add(tf.keras.layers.Dense(nodes_at_layers, activation = 'relu'))
        model_name += f'dense{nodes_at_layers}_'

    model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
    model._name = model_name[:-1]

    models.append(model)

In [12]:
models[0].summary()

Model: "dense64_dense64_dense64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9,217
Trainable params: 9,217
Non-trainable params: 0
_________________________________________________________________


## Optimization function

In [15]:
def get_models(num_layers: int,
    min_nodes_per_layer: int,
    max_nodes_per_layer: int,
    node_step_size: int,
    input_shape: tuple,
    hidden_layer_activation: str = 'relu',
    num_nodes_at_output: int = 1,
    output_layer_activation: str = 'sigmoid') -> list:
    node_options = list(range(min_nodes_per_layer, max_nodes_per_layer + 1, node_step_size))
    layer_possibilities = [node_options] * num_layers
    layer_node_permutations = list(itertools.product(*layer_possibilities))

    models = []
    for permutation in layer_node_permutations:
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.InputLayer(input_shape = input_shape))
        model_name = ''

        for nodes_at_layer in permutation:
            model.add(tf.keras.layers.Dense(nodes_at_layers, activation = hidden_layer_activation))
            model_name += f'dense{nodes_at_layer}_'

        model.add(tf.keras.layers.Dense(num_nodes_at_output, activation = output_layer_activation))
        model._name = model_name[:-1]
        models.append(model)
    
    return models

In [105]:
all_models = get_models(
    num_layers=3,
    min_nodes_per_layer=5,
    max_nodes_per_layer=15,
    node_step_size=5,
    input_shape=(14,)
)

In [106]:
for model in all_models:
    print(model.name)

dense5_dense5_dense5
dense5_dense5_dense10
dense5_dense5_dense15
dense5_dense10_dense5
dense5_dense10_dense10
dense5_dense10_dense15
dense5_dense15_dense5
dense5_dense15_dense10
dense5_dense15_dense15
dense10_dense5_dense5
dense10_dense5_dense10
dense10_dense5_dense15
dense10_dense10_dense5
dense10_dense10_dense10
dense10_dense10_dense15
dense10_dense15_dense5
dense10_dense15_dense10
dense10_dense15_dense15
dense15_dense5_dense5
dense15_dense5_dense10
dense15_dense5_dense15
dense15_dense10_dense5
dense15_dense10_dense10
dense15_dense10_dense15
dense15_dense15_dense5
dense15_dense15_dense10
dense15_dense15_dense15


In [107]:
print(len(all_models))

27


## Training Function


In [108]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [109]:
def optimize(models: list,
             X_train: np.array,
             y_train: np.array,
             X_test: np.array,
             y_test: np.array,
             epochs: int = 50,
             verbose: int = 0) -> pd.DataFrame:
    
    results = []

    def train(model: tf.keras.Sequential) -> dict:
        model.compile(
            loss=tf.keras.losses.binary_crossentropy,
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
        )
        model.fit(X_train, y_train, epochs = epochs, verbose = verbose)
        preds = model.predict(X_test)
        prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(preds)]

        return {
            'model_name': model.name,
            'test_accuracy': accuracy_score(y_test, prediction_classes),
            'test_precision': precision_score(y_test, prediction_classes),
            'test_recall': recall_score(y_test, prediction_classes),
            'test_f1': f1_score(y_test, prediction_classes),
        }
    
    for model in models:
        try:
            print(model.name, end='...')
            res = train(model=model)
            results.append(res)
        except Exception as e:
            print(f'{model.name} --> {str(e)}')

    return pd.DataFrame(results)

In [110]:
dados_entrada.keys()
df = dados_entrada.copy(deep=True)
df = df.dropna()

df['RoasHigh'] = [1 if yvar >= 0.3 else 0 for yvar in df['yvar']]

df.drop(['views_5s_x_days', 'yvar', 'publico_cat_AdvShp', 'publico_cat_Amplo',
       'publico_cat_erro_taxonomia', 'publico_cat_Interesses',
       'publico_cat_Reeng', 'publico_cat_Rmkt', 'formato_ad_Carr',
       'formato_ad_erro_taxonomia', 'formato_ad_Est', 'formato_ad_Gif'], axis = 1, inplace=True)

In [111]:
from sklearn.model_selection import train_test_split

X = df.drop('RoasHigh', axis = 1)
y = df['RoasHigh']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
%%time

optimization_results = optimize(
    models = all_models,
    X_train = X_train,
    y_train = y_train,
    X_test = X_test,
    y_test = y_test
)

5/5 [==============================] - 0s 1ms/step
Wall time: 1min 18s


In [113]:
optimization_results.sort_values(by = 'test_accuracy', ascending=False)

,model_name,test_accuracy,test_precision,test_recall,test_f1
18,dense15_dense5_dense5,0.952381,0.833333,0.454545,0.588235
15,dense10_dense15_dense5,0.952381,0.833333,0.454545,0.588235
13,dense10_dense10_dense10,0.945578,0.714286,0.454545,0.555556
25,dense15_dense15_dense10,0.945578,0.714286,0.454545,0.555556
24,dense15_dense15_dense5,0.945578,0.714286,0.454545,0.555556
4,dense5_dense10_dense10,0.945578,0.714286,0.454545,0.555556
5,dense5_dense10_dense15,0.945578,0.714286,0.454545,0.555556
6,dense5_dense15_dense5,0.945578,0.714286,0.454545,0.555556
23,dense15_dense10_dense15,0.945578,0.714286,0.454545,0.555556
22,dense15_dense10_dense10,0.945578,0.714286,0.454545,0.555556
